![](https://i.ytimg.com/vi/j4XvJEbaVJg/maxresdefault.jpg)

## Импортируем библиотеки

- `pandas` - для удобной работы работы с данными в табличном представлении 
- `seaborn` - библиотека для визуализации данных и анализа
- `matplotlib` - тоже библиотека для визуализыции
- `warnings` - для игнорирования предупреждений




In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Данные будем брать из открытого репозитория 

In [ ]:
# Загрузка данных из репозитория
df=pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')

In [ ]:
#Посмотрим на первые пять строк нашего датасета
df.head()

Приведем дату к типу `data` 

In [ ]:
df['date']=pd.to_datetime(df['date'])

Посмотрим типы данных в колонках 


In [ ]:
df.info()

Посмотрим на количество пустых значений в наших данных

In [ ]:
#Убираем ограничение на вы
pd.set_option('display.max_rows',None)
df.isnull().sum()

In [ ]:
pd.set_option('display.max_rows',10)

Посмотрим на размерность наших данных

In [ ]:
df.shape

Все данные при парсинге из репозитория были включены. Размерность набора данных составляет `191147 строк` и `67 столбцов`

Дополним набор данных новой информацией, чтобы в случае необходимости точность модели при обучении была больше. Также новые данные могут пригодиться для того, чтобы включить их в аналих данных, из которого можно вынести некоторые зависимости, если они присутствуют 

В качестве новых данных возьмём среднее количество death и заражённых на регион.

In [ ]:
# Заполнение пустых полей
df[['location', 'new_cases', 'new_deaths']]=df[['location', 'new_cases', 'new_deaths']].fillna(0)

# Формирование дополнительных атрибутов
grouped_cases=df[['location', 
                  'new_cases', 
                  'new_deaths']].groupby(by="location").mean().rename(columns={'new_cases':'mean_new_cases', 
                                                                               'new_deaths':'mean_new_deaths'})

In [ ]:
grouped_cases.head()

Сконкатенируем полученный набор данных с исходным

In [ ]:
df=df.merge(grouped_cases, on='location')
df.head()

На данном этапе мы сгенерировали новые признаки для нашего набора данных

---

Для более удобного взаимодействия с нашими данными заполним пустые значения нулями

In [ ]:
df=df.fillna(0)

### Определение наиболее значимых атрибутов
Чтобы найти наиболее значимые атрибуты, построим корреляцию Пирсона на тепловой карте

In [ ]:
# Фомирование корреляции Пирсона
corr=df.corr()
plt.figure(figsize=(70, 70))

heatmap = sns.heatmap(corr, vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':20}, pad=20)

Как видим выше, достаточно много признаков имеют высокий коэффициент корреляции, но наиболее значимыми атрибутами являются: `total_casem, new_case, new_cases_smoothed, total_deaths, new_deaths и new_deaths_smoothed`

---

In [ ]:
# Выведем все колонки и типы данных в них для описания итогового набора данных
df.info()

`total_cases` - Всего подтвержденных случаев COVID-19. Подсчеты могут включать вероятные случаи, о которых сообщается.

`new_cases` - Новые подтвержденные случаи заболевания COVID-19. Подсчеты могут включать вероятные случаи, о которых сообщается. В редких случаях, когда наш источник сообщает об отрицательном ежедневном изменении из-за корректировки данных, мы устанавливаем для этой метрики значение NAME.

`new_cases_smoothed` - Новые подтвержденные случаи COVID-19 (7-дневный период сглажен). Подсчеты могут включать вероятные случаи, о которых сообщается.

`total_cases_per_million` - Общее количество подтвержденных случаев COVID-19 на 1 000 000 человек. Подсчеты могут включать вероятные случаи, о которых сообщается.

`new_cases_per_million` - Новые подтвержденные случаи COVID-19 на 1 000 000 человек. Подсчеты могут включать вероятные случаи, о которых сообщается.

`new_cases_smoothed_per_million` - Новые подтвержденные случаи COVID-19 (сглаженные за 7 дней) на 1 000 000 человек. Подсчеты могут включать вероятные случаи, о которых сообщается.

`total_deaths` - Общее число смертей, связанных с COVID-19. Сообщалось, что подсчеты могут включать вероятные смертельные случаи.

`new_deaths` - Новые случаи смерти, связанные с COVID-19. Сообщалось, что подсчеты могут включать вероятные смертельные случаи. В редких случаях, когда наш источник сообщает об отрицательном ежедневном изменении из-за корректировки данных, мы устанавливаем для этой метрики значение NAME.

`new_deaths_smoothed` - Новые случаи смерти, связанные с COVID-19 (7-дневный период сглажен). Сообщалось, что подсчеты могут включать вероятные смертельные случаи.

`total_deaths_per_million` - Общее число смертей, связанных с COVID-19, на 1 000 000 человек. Сообщалось, что подсчеты могут включать вероятные смертельные случаи.

`new_deaths_per_million` - Новые случаи смерти, связанные с COVID-19, на 1 000 000 человек. Сообщалось, что подсчеты могут включать вероятные смертельные случаи.

`new_deaths_smoothed_per_million` - Новые случаи смерти, связанные с COVID-19 (сглаженные за 7 дней) на 1 000 000 человек. Сообщалось, что подсчеты могут включать вероятные смертельные случаи.

`excess_mortality` - Процентная разница между зарегистрированным числом еженедельных или ежемесячных смертей в 2020-2021 годах и прогнозируемым числом смертей за тот же период на основе предыдущих лет. Для получения дополнительной информации см. https://github.com/owid/covid-19-data/tree/master/public/data/excess_mortality 

`excess_mortality_cumulative` - Процентная разница между совокупным числом смертей с 1 января 2020 года и совокупным прогнозируемым числом смертей за тот же период, основанным на предыдущих годах. Для получения дополнительной информации см. https://github.com/owid/covid-19-data/tree/master/public/data/excess_mortality

`excess_mortality_cumulative_absolute` - Совокупная разница между зарегистрированным числом смертей с 1 января 2020 года и прогнозируемым числом смертей за тот же период на основе предыдущих лет. Для получения дополнительной информации см. https://github.com/owid/covid-19-data/tree/master/public/data/excess_mortality избыток

`excess_mortality_cumulative_per_million`- Совокупная разница между зарегистрированным числом смертей с 1 января 2020 года и прогнозируемым числом смертей за тот же период на основе предыдущих лет на миллион человек. Для получения дополнительной информации см. https://github.com/owid/covid-19-data/tree/master/public/data/excess_mortality

`icu_patients` - Количество пациентов с COVID-19 в отделениях интенсивной терапии (ОИТ) в данный день

`icu_patients_per_million` - Количество пациентов с COVID-19 в отделениях интенсивной терапии (ОИТ) в данный день на 1 000 000 человек

`hosp_patients` - Количество пациентов с COVID-19 в больнице в данный день

`hosp_patients_per_million` - Количество пациентов с COVID-19, находящихся в больнице в данный день на 1 000 000 человек

`weekly_icu_admissions` - Число пациентов с COVID-19, впервые поступивших в отделения интенсивной терапии (ОИТ) за данную неделю

`weekly_icu_admissions_per_million` - Число пациентов с COVID-19, впервые поступивших в отделения интенсивной терапии (ОИТ) за данную неделю на 1 000 000 человек

`weekly_hosp_admissions` - Число пациентов с COVID-19, впервые поступивших в больницы за данную неделю

`weekly_hosp_admissions_per_million` - Число пациентов с COVID-19, впервые поступивших в больницы за данную неделю на 1 000 000 человек

`stringency_index` - Индекс строгости реагирования правительства: сводный показатель, основанный на 9 показателях реагирования, включая закрытие школ, закрытие рабочих мест и запреты на поездки, масштабированный до значения от 0 до 100 (100 = самый строгий ответ)

`reproduction_rate` - Оценка в реальном времени эффективной скорости размножения (R) COVID-19. https://github.com/crondonm/TrackingR/tree/main/Estimates-Database

`total_tests` - Общее количество тестов на COVID-19

`new_tests` - Новые тесты на COVID-19 (рассчитываются только для последовательных дней)

`total_tests_per_thousand` - Общее количество тестов на COVID-19 на 1000 человек

`new_tests_per_thousand` - Новые тесты на COVID-19 на 1000 человек

`new_tests_smoothed` - Новые тесты на COVID-19 (7-дневный сглаженный). Для стран, которые не сообщают данные тестирования на ежедневной основе, мы предполагаем, что тестирование менялось одинаково ежедневно в течение любых периодов, в течение которых данные не сообщались. Это дает полный ряд ежедневных показателей, которые затем усредняются в течение скользящего 7-дневного окна

`new_tests_smoothed_per_thousand` - Новые тесты на COVID-19 (7-дневный сглаженный) на 1000 человек

`positive_rate` - Доля положительных тестов на COVID-19, приведенная в виде скользящего среднего за 7 дней (это обратная величина tests_per_case)

`tests_per_case` - Тесты, проведенные для каждого нового подтвержденного случая COVID-19, приведены в виде скользящего среднего значения за 7 дней (это обратная величина positive_rate)

`tests_units` - Единицы измерения, используемые местоположением для представления своих данных тестирования

`total_vaccinations` - Общее количество доз вакцинации против COVID-19, введенных вакцинированным людям

`people_vaccinated` - Общее число людей, получивших хотя бы одну дозу вакцины

`people_fully_vaccinated` - Общее число людей, получивших все дозы, предписанные первоначальным протоколом вакцинации 

`total_boosters` - Общее количество введенных бустерных доз вакцинации против COVID-19 (дозы, введенные сверх количества, предписанного протоколом вакцинации)

`new_vaccinations` - Вводимые новые дозы вакцинации против COVID-19 (рассчитываются только для последовательных дней)

`new_vaccinations_smoothed` - Введены новые дозы вакцинации против COVID-19 (7-дневный период сглажен). Для стран, которые не сообщают данные о вакцинации на ежедневной основе, мы предполагаем, что вакцинация менялась одинаково ежедневно в течение любых периодов, в течение которых данные не сообщались. Это дает полный ряд ежедневных показателей, которые затем усредняются в течение скользящего 7-дневного окна

`total_vaccinations_per_hundred` - Общее количество доз вакцинации против COVID-19, введенных на 100 человек в общей популяции

`people_vaccinated_per_hundred` - Общее число людей, получивших по крайней мере одну дозу вакцины на 100 человек в общей численности населения

`people_fully_vaccinated_per_hundred`- Общее число людей, получивших все дозы, предписанные протоколом первичной вакцинации, на 100 человек в общей численности населения

`total_boosters_per_hundred` - Общее количество бустерных доз вакцинации против COVID-19, введенных на 100 человек в общей численности населения

`new_vaccinations_smoothed_per_million` - Новые дозы вакцинации против COVID-19, введенные (сглаженные на 7 дней) на 1 000 000 человек от общей численности населения

`new_people_vaccinated_smoothed` - Ежедневное количество людей, получающих свою первую дозу вакцины (сглажено на 7 дней)

`new_people_vaccinated_smoothed_per_hundred` - Ежедневное число людей, получающих первую дозу вакцины (7-дневная сглаженная) на 100 человек в общей численности населения

`iso_code` - ISO 3166-1 alpha-3 – трехбуквенные коды стран

`continent` - Континент географического местоположения

`location` - Географическое положение

`date` - Дата наблюдения

`population` - Численность населения (последние доступные значения). https://github.com/owid/covid-19-data/blob/master/scripts/input/un/population_latest.csv для получения полного списка источников

`population_density` - Количество людей, разделенное на площадь земли, измеренную в квадратных километрах, самый последний год, доступный

`median_age` - Средний возраст населения, прогноз ООН на 2020 год

`aged_65_older` - Доля населения в возрасте 65 лет и старше, последний доступный год

`aged_70_older` - Доля населения в возрасте 70 лет и старше в 2015 году 

`gdp_per_capita` - Валовой внутренний продукт по паритету покупательной способности (постоянный международный доллар 2011 года), последний доступный год

`extreme_poverty` - Доля населения, живущего в крайней нищете, последний год, доступный с 2010 года 

`cardiovasc_death_rate` - Уровень смертности от сердечно-сосудистых заболеваний в 2017 году (годовое число смертей на 100 000 человек)
`diabetes_prevalence` - Распространенность диабета (% населения в возрасте от 20 до 79 лет) в 2017 году

`female_smokers` - Доля курящих женщин, за последний год доступных

`male_smokers` - Доля курящих мужчин, за последний год доступных

`handwashing_facilities` - Доля населения, имеющего базовые средства для мытья рук в помещениях, за последний год количество доступных

`hospital_beds_per_thousand` - Больничные койки на 1000 человек, последний доступный год с 2010 года

`life_expectancy` - Ожидаемая продолжительность жизни при рождении в 2019 году

`human_development_index` - Сводный индекс, измеряющий средние достижения в трех основных аспектах человеческого развития — долгая и здоровая жизнь, знания и достойный уровень жизни. Значения за 2019 год, импортированные из http://hdr.undp.org/en/indicators/137506

## Пустые значения
Пустые значения были предобработанны ранее, после предобработки их больше не осталось

In [ ]:
pd.set_option('display.max_rows',None)
df.isnull().sum()

In [ ]:
pd.set_option('display.max_rows',10)

### Плотность распределения данных
Сформируем графики плотностей для каждого признака

Для начала выведем графики распределения категориальных признаков, потом напишем функцию для быстрого расчета графиков распределения числовых переменных

In [ ]:
#выведу тут пять строк из набора данных для удобства построения графиков
pd.options.display.max_columns = None

df.head()

In [ ]:
# Распределение iso-code ()
plt.figure(figsize=(10, 5))
sns.kdeplot(df['iso_code'].value_counts())
plt.title('Плотность iso_code')
plt.xlabel('Значения')
plt.ylabel('Распределение')
plt.show()

In [ ]:
# Распределение континентов
plt.figure(figsize=(10, 5))
sns.kdeplot(df['continent'].value_counts())
plt.title('Distribution continent')
plt.xlabel('Значения')
plt.ylabel('Распределение')
plt.show()

In [ ]:
# Распределение географического положения
plt.figure(figsize=(10, 5))
sns.kdeplot(df['location'].value_counts())
plt.title('Distribution location')
plt.xlabel('Значение')
plt.ylabel('Распределение')
plt.show()

In [ ]:
#Функция вывода распределения каждого атрибута, являющегося численным признаком
def plot(column):
    plt.figure(figsize=(10, 5))
    sns.kdeplot(df[column])
    plt.title('Distribution '+column)
    plt.xlabel('Значения')
    plt.ylabel('Распределение')
    plt.show()

In [ ]:
#Применение функции
for column in df[:100].select_dtypes(exclude=['object']).columns:
    plot(column)

---

In [ ]:
#Алгоритм вычисления нового атрибута Rt
df['Rt']=None
data=pd.DataFrame()
for country in df['location'].value_counts().keys():
    r=df[df['location']==country].copy()
    da=pd.DataFrame()
    for i in range(0, len(r), 8):
        tida=pd.DataFrame()
        su=r['new_cases'].tail(8).tail(4).sum()/r['new_cases'].tail(8).head(4).sum()
        tida=r.tail(8)
        tida['Rt']=su
        r.drop(r.tail(8).index,inplace=True)
        da=da.append(tida)
    data=data.append(da)

Т.к наш новый признак может дать на выход ну`левое значение, заменим их нулями


In [ ]:
data = data.fillna(0)

In [ ]:
#Воссоздадим индексы 
data.reset_index(drop=True, inplace=True)
#Теперь, в качестве df Датафрема передадим значения data
df=data

In [ ]:
df.head(10)

**Результат алгоритма выше демонстрирует формирование дополнительного атрибута Rt - коэффициент распространения инфекции**

### Анализ возможности определения изменения эпидемиологической
Проанализируем полученные данные для нескольких стран

In [ ]:
d=pd.DataFrame({'Russia': [list(df[df['location']=='Russia']['Rt'])[0]], 
                'Mexico':[list(df[df['location']=='Mexico']['Rt'])[0]], 
                'France': [list(df[df['location']=='France']['Rt'])[0]], 
                'Taiwan':[list(df[df['location']=='Taiwan']['Rt'])[0]], 
                'United States':[list(df[df['location']=='United States']['Rt'])[0]], 
                'Japan':[list(df[df['location']=='Japan']['Rt'])[0]], 
                'Canada':[list(df[df['location']=='Canada']['Rt'])[0]], 
                'Singapore':[list(df[df['location']=='Singapore']['Rt'])[0]],}).T

Выведем актуальные Rt - значения для каждой страны

In [ ]:
plt.rcParams.update({'font.size': 15,})
plt.figure(figsize=(15, 8))
plots = sns.barplot(x=d.index, y=d[0], data=df)

for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=15, xytext=(0, 8),
                   textcoords='offset points')

plt.title('Анализ эпидемиологической обстановки')
plt.ylabel('Rt - значение')
plt.xlabel('Страна')
plt.show()

Из полученных результатов можно сказать, что максимальное Rt - значение предложенных стран - это США`(2.13)`. Минимальное - Мексика `(0.0)`. Россия на текущий день имеет Rt значение `0.87` 

---

* Парсинг данных - Набор данных загружен из репозитория, а также произведенно дополнение некоторыми атрибутами
* Предобработка данных и выделение значимых атрибутов - Набор данных обработан от пустых значений, а также определенны наиболее "важные" атрибуты по корреляции Пирсона
* Описание структуры набора данных - для каждого атрибута представленно описание и плотность распределения данных
* Формирование дополнительных атрибутов - Сформированн дополнительный атрибут Rt, который определяет коэффициента распространения инфекции за последние 8 дней

In [ ]:
# Сохранение данных
df.to_csv('result_data.csv', encoding='utf-8-sig', index=False)

# PART TWO

## Импортируем библиотеки

- `pandas` - для удобной работы работы с данными в табличном представлении 
- `numpy` - для работы с матрицами и быстрого выполнения математических опреций, а также использования математических функций в библиотеке

In [ ]:
import pandas as pd
import numpy as np

---

Прочитаем набор данных 

In [ ]:
df=pd.read_csv('result_data.csv')

### Стратификация
Перед разделением стратифицируем данные, чтобы получить одинаковую в процентом соотношении выборку, чтобы не было перевеса на какой-то один класс и такая ситуация не повлияла на некорректное обучение модели

In [ ]:
df

In [ ]:
#уберем из набора данных значения которых бесконечно 
df.replace([np.inf, -np.inf], np.nan, inplace=True)

Уберем из набора данных строки с избыточным значением RT, для того, чтобы веса модели были правильно определены, т.к значения выше среднего порога очень сильно влияют на построение весов

In [ ]:
df=df[df['Rt']<5].reset_index(drop=True)

### Определение переменной опасности

Определим целевую переменную как `Danger`, эта переменная будет коррелировать с нашим `Rt`

In [ ]:
df1=df[df['Rt']<=0.7]
df1['Danger']=0

In [ ]:
df2=df[(df['Rt']>0.7) & (df['Rt']<=0.95)]
df2['Danger']=1

In [ ]:
df3=df[df['Rt']>0.95]
df3['Danger']=2

Сконкатенируем данные опасности с нашим набором данных

In [ ]:
df=pd.concat([df1, df2, df3]).reset_index(drop=True)

Для построения намбора данных будем использовать три фичи `new_cases, new_deaths, Rt`

In [ ]:
X=df[['new_cases', 'new_deaths', 'Rt']]
y=df['Danger']

Разобъём набор данных таким образом, как это рекомендовано согласно документации `Sklearn`. А именно `30 на 70`. Как представленно в описании, такая выборка является оптимальной, поскольку абсолютное большинство данных должно находится при обучении модели, чтобы получить наиболее оптимизированную модель со стороны её точности

In [ ]:
#Получение выборок
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

---

 Классифиткация 

Рассмотрим три модели классификации
### KNeighborsClassifier
Классификация на основе соседей - это тип обучения на основе экземпляров или необобщающего обучения: он не пытается построить общую внутреннюю модель, а просто сохраняет экземпляры обучающих данных. Классификация вычисляется простым большинством голосов ближайших соседей каждой точки: точке запроса назначается класс данных, который имеет наибольшее количество представителей среди ближайших соседей точки.

### RandomForestClassifier
Случайный лес — это метаоценка, которая соответствует ряду классификаторов дерева решений для различных подвыборок набора данных и использует усреднение для повышения точности прогнозирования и контроля переобучения. Размер подвыборки управляется параметром max_samples, если bootstrap=True (по умолчанию), в противном случае для построения каждого дерева используется весь набор данных
### GaussianNB
Наи́вный ба́йесовский классифика́тор — простой вероятностный классификатор, основанный на применении теоремы Байеса со строгими (наивными) предположениями о независимости. В зависимости от точной природы вероятностной модели, наивные байесовские классификаторы могут обучаться очень эффективно

## Матрикики
Рассмотрим две метрикики для оценивания модели классификации

### accuracy f1-score
Это гармоническое среднее значений точности и полноты. Возьмём её, потому что она дает лучшую оценку неправильно классифицированных случаев

### macro avg f1-score

macro avg f1-score пожалуй, самый простой из многочисленных методов усреднения. Макроусредненная оценка F1 (или макрооценка F1) вычисляется путем взятия среднего арифметического (также известного как невзвешенное среднее) всех оценок F1 для каждого класса. Этот метод будет взят, поскольку он обрабатывает все классы одинаково, независимо от их значений поддержки

---

Обучение


In [ ]:
#Импорт моделей
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
#Обучение
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
preds=neigh.predict(X_test)
print(classification_report(preds, y_test))

In [ ]:
#Обучение
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_preds=rfc.predict(X_test)
print(classification_report(rfc_preds, y_test))

In [ ]:
#Обучение
gnb = GaussianNB()
gnb.fit(X_train, y_train)
gnb_preds=gnb.predict(X_test)
print(classification_report(gnb_preds, y_test))

### Вывод
Наиболее оптимальной моделью будет `KNeighborsClassifier` c accuracy f1-score = `0.78` и macro avg f1-score = `0.74`, поскольку по сравнению с другими он показал наилучший результат. `RandomForestClassifier` не будет взят, поскольку у него явное переобучение

---

* Разбиение набора данных - набор данныхз разбит на обучаюшую и тестовую выборки
* Классификация - выбраны 3 алгоритма классификации
* Обучение - произведена классификация по уровню опасности

In [ ]:
# Сохранение данных
df.to_csv('result_data.csv', encoding='utf-8-sig', index=False)